# Task4

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Subject

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-08-11_00-39-35/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-08-11_00-39-35

## Best metric (RMSE)

179.106

1.005

In [2]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.LSTM'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_subject'
    }

In [3]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='metric',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='metric',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='metric',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-08-11 03:31:19,197] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-08-11 00:39:38,322	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
2023-08-11 00:39:40,542	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-08-11 00:39:40,613	INFO wandb.py:320 -- Already logged into W&B.
(_WandbLoggingActor pid=181672) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=181672) wandb: wandb version 0.15.8 is available!  To upgrade, please run:
(_WandbLoggingActor pid=181672) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=181672) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=181672) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35/FSR_Trainable_e5ebe0c9_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,imputer=sklearn_impute_Simpl_2023-08-11_00-39-40/wandb/run-20230811_003952-e5ebe0c9
(_WandbLoggingActor pid=181672) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=181672) wandb: Syncing run FSR_Trainable_e5ebe0c9
(_WandbLoggingActor pid=181672) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_

Trial name,date,done,hostname,iterations_since_restore,mae_coord,mae_force,mape_coord,mape_force,metric,node_ip,pid,rmse_coord,rmse_force,time_since_restore,time_this_iter_s,time_total_s,timestamp,tmae_coord,tmae_force,tmape_coord,tmape_force,training_iteration,trial_id,trmse_coord,trmse_force
FSR_Trainable_0263d11d,2023-08-11_00-46-53,True,DESKTOP-0P789CI,1,3.83395,1243.99,1.10314,4.96748e+08,6.20889,172.26.215.93,188629,2.35646,829.21,3.17325,3.17325,3.17325,1691682413,7.28164,2.9087,5.22389e+14,32.867,1,0263d11d,4.48525,1.72364
FSR_Trainable_043a5c07,2023-08-11_00-47-00,True,DESKTOP-0P789CI,64,5.56496,718.046,1.27169,9.01965e+17,0.715225,172.26.215.93,181673,2.65691,480.161,400.072,6.67965,400.072,1691682420,1.18188,0.29724,7.81235e+13,4.43445e+14,64,043a5c07,0.526588,0.188638
FSR_Trainable_08dbc938,2023-08-11_01-00-41,True,DESKTOP-0P789CI,1,5.49974,1272.08,1.54442,7.11904e+17,42.3578,172.26.215.93,194170,2.56798,941.895,16.8932,16.8932,16.8932,1691683241,34.9217,10.2377,2.23357e+16,2.46074e+16,1,08dbc938,35.2412,7.1166
FSR_Trainable_08ece1c7,2023-08-11_00-54-15,True,DESKTOP-0P789CI,8,4.95041,1422.36,1.23723,2.77166e+18,0.831566,172.26.215.93,191403,2.56307,943.166,55.0824,6.38127,55.0824,1691682855,1.08289,0.513415,8.25355e+13,1.07223e+15,8,08ece1c7,0.528459,0.303107
FSR_Trainable_0daad9a2,2023-08-11_00-50-09,True,DESKTOP-0P789CI,32,4.79424,1045.58,1.27489,1.75673e+18,0.749347,172.26.215.93,189324,2.49671,726.741,152.569,4.51588,152.569,1691682609,1.03638,0.390066,8.08905e+13,7.04061e+14,32,0daad9a2,0.505578,0.243769
FSR_Trainable_12d3f88c,2023-08-11_01-10-33,True,DESKTOP-0P789CI,100,5.10608,660.407,1.19581,7.7794e+17,0.66628,172.26.215.93,199460,2.53949,463.226,116.262,1.56638,116.262,1691683833,1.07379,0.269049,7.55217e+13,4.0038e+14,100,12d3f88c,0.495648,0.170632
FSR_Trainable_14a7a158,2023-08-11_01-05-16,True,DESKTOP-0P789CI,100,5.34303,674.391,1.18652,8.19509e+17,0.679719,172.26.215.93,196148,2.58393,485.066,101.278,0.914241,101.278,1691683516,1.12622,0.265228,7.34211e+13,3.85182e+14,100,14a7a158,0.506162,0.173557
FSR_Trainable_170adff7,2023-08-11_01-06-26,True,DESKTOP-0P789CI,100,5.31857,632.744,1.17394,7.06356e+17,0.672783,172.26.215.93,196857,2.58986,467.646,101.114,0.877878,101.114,1691683586,1.12069,0.250942,7.45695e+13,3.52458e+14,100,170adff7,0.506761,0.166022
FSR_Trainable_19e6058b,2023-08-11_00-41-27,True,DESKTOP-0P789CI,4,27.0655,2613.88,4.03607,3.16548e+18,2.39461,172.26.215.93,183244,8.96946,1692.64,12.1478,3.24274,12.1478,1691682087,5.8739,0.932246,2.41018e+13,1.37614e+15,4,19e6058b,1.87343,0.521179
FSR_Trainable_1a98df0c,2023-08-11_00-57-35,True,DESKTOP-0P789CI,100,5.2624,659.165,1.17718,7.52539e+17,0.683731,172.26.215.93,191593,2.6214,493.077,230.736,2.39619,230.736,1691683055,1.10912,0.25361,7.38813e+13,3.30162e+14,100,1a98df0c,0.509632,0.174099


(_WandbLoggingActor pid=181836) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=181836) wandb: wandb version 0.15.8 is available!  To upgrade, please run:
(_WandbLoggingActor pid=181836) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=181836) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=181836) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35/FSR_Trainable_043a5c07_2_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,imputer=sklearn_impute_Simpl_2023-08-11_00-39-46/wandb/run-20230811_004002-043a5c07
(_WandbLoggingActor pid=181836) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=181836) wandb: Syncing run FSR_Trainable_043a5c07
(_WandbLoggingActor pid=181836) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=181836) wandb: 🚀 View run at https://wandb.ai/se

TuneError: The Ray Tune run failed. Please inspect the previous error messages for a cause. After fixing the issue, you can restart the run from scratch or continue this run. To continue this run, you can use `tuner = Tuner.restore("/home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35", trainable=...)`.

In [4]:
results = tuner.restore(
    path='/home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35',
    trainable=FSR_Trainable
).fit()

Failed to read the results for 2 trials:
- /home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35/FSR_Trainable_48eada07_88_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,imputer=sklearn_impute_Simp_2023-08-11_01-12-28
- /home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35/FSR_Trainable_57b5d6f7_89_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,imputer=sklearn_impute_Simp_2023-08-11_01-45-46
2023-08-11 03:31:22,078	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-08-11 03:31:23,247	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-08-11 03:31:23,301	INFO experiment_state.py:435 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2023-08-11 03:31:23,307	WARNING trial_runner.py:418 -- Attempting to resume experiment from /home/seokj/ray_results/FSR_Trainable_2023-08-11_00-39-35. This will ignore any new changes to the specification.
2023-08-11 03:31:23,308	INFO trial_runner.py:422 -- Using the newest experiment state file found within the experiment directory: experiment_state-2023-08-11_01-45-30.json
2023-08-11 03:31:23,612	INFO wandb.py:320 -- Already logged into W&B.
(FSR_Trainable pid=209203) 2023-08-11 03:31:32,992	INFO trainable.py:918 -- Restored on 172.26.215.93 from checkpoint: /tmp/checkpoint_tmp_76d6e12af35e42dda908e5b68b809d77
(FSR_Trainable pid=209203) 2023-08-11 03:31:32,993	INFO trainable.py:927 -- Current state after restoring: {'_iteration': 55, '_timesteps_total': None, '_time_total': 65.34051585197449, '_episodes_total': None}
/home/seokj/wo

Trial name,date,done,hostname,iterations_since_restore,mae_coord,mae_force,mape_coord,mape_force,metric,node_ip,pid,rmse_coord,rmse_force,time_since_restore,time_this_iter_s,time_total_s,timestamp,tmae_coord,tmae_force,tmape_coord,tmape_force,training_iteration,trial_id,trmse_coord,trmse_force
FSR_Trainable_0bca8ed6,2023-08-11_03-33-24,True,DESKTOP-0P789CI,4,5.83276,868.916,1.55563,1.66923e+18,0.733683,172.26.215.93,212252,2.59852,513.723,2.71943,0.558843,2.71943,1691692404,1.22675,0.396802,8.80448e+13,9.10261e+14,4,0bca8ed6,0.517768,0.215915
FSR_Trainable_0e4322aa,2023-08-11_03-32-58,True,DESKTOP-0P789CI,1,7.578,1651.95,1.73146,3.78392e+18,1.01374,172.26.215.93,211539,3.15538,970.318,1.31382,1.31382,1.31382,1691692378,1.74873,0.623174,9.26633e+13,1.53126e+15,1,0e4322aa,0.694225,0.319515
FSR_Trainable_119b1b07,2023-08-11_03-33-15,True,DESKTOP-0P789CI,4,5.67262,858.792,1.49728,1.58982e+18,0.710309,172.26.215.93,211941,2.54437,504.846,2.8806,0.65383,2.8806,1691692395,1.18513,0.382535,8.59213e+13,8.56557e+14,4,119b1b07,0.505472,0.204837
FSR_Trainable_21fdf89c,2023-08-11_03-32-32,True,DESKTOP-0P789CI,45,5.32435,651.537,1.19473,8.34275e+17,0.675416,172.26.215.93,209203,2.60022,453.129,37.8405,0.730253,103.181,1691692352,1.11918,0.263298,7.2956e+13,4.10181e+14,100,21fdf89c,0.507117,0.168299
FSR_Trainable_28ec7133,2023-08-11_03-31-58,True,DESKTOP-0P789CI,2,5.30413,1140.46,1.38129,1.93086e+18,0.784679,172.26.215.93,210044,2.50295,840.436,3.46317,1.38829,3.46317,1691692318,1.14801,0.403903,9.4161e+13,6.96675e+14,2,28ec7133,0.515151,0.269529
FSR_Trainable_35aed758,2023-08-11_03-32-22,True,DESKTOP-0P789CI,1,20.5306,1862.51,3.13937,2.78963e+18,1.87748,172.26.215.93,210732,7.2083,1199.9,1.42322,1.42322,1.42322,1691692342,4.43144,0.749304,3.34478e+13,1.42901e+15,1,35aed758,1.46896,0.408525
FSR_Trainable_3838379f,2023-08-11_03-32-42,True,DESKTOP-0P789CI,60,5.41718,643.069,1.17597,7.70419e+17,0.674217,172.26.215.93,209204,2.62062,465.954,45.312,0.679527,89.8499,1691692362,1.13167,0.255212,7.56335e+13,3.84505e+14,100,3838379f,0.508349,0.165868
FSR_Trainable_46a66fbd,2023-08-11_03-32-33,True,DESKTOP-0P789CI,1,19.1657,1342.19,3.33368,2.21604e+18,1.73027,172.26.215.93,210972,6.91445,945.257,1.1758,1.1758,1.1758,1691692353,4.20424,0.499856,6.46556e+13,9.48777e+14,1,46a66fbd,1.42271,0.307553
FSR_Trainable_48eada07,2023-08-11_03-31-59,True,DESKTOP-0P789CI,4,5.55037,902.659,1.50565,1.6327e+18,0.742966,172.26.215.93,209206,2.53547,538.484,8.40614,1.55633,8.40614,1691692319,1.17553,0.417023,8.88168e+13,9.11523e+14,4,48eada07,0.507821,0.235144
FSR_Trainable_4f7002fa,2023-08-11_03-32-43,True,DESKTOP-0P789CI,1,19.6853,1776.34,2.85575,3.48467e+18,1.84951,172.26.215.93,211207,6.59445,1079.48,1.17402,1.17402,1.17402,1691692363,4.36987,0.759864,3.43276e+13,1.6675e+15,1,4f7002fa,1.42126,0.428248


/home/seokj/workspace/.venv/lib/python3.10/site-packages/optuna/trial/_trial.py:494: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
(_WandbLoggingActor pid=209481) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(FSR_Trainable pid=209205) 2023-08-11 03:31:33,552	INFO trainable.py:918 -- Restored on 172.26.215.93 from checkpoint: /tmp/checkpoint_tmp_ead7f66d7d38485bb5534b922360ae07 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(FSR_Trainable pid=209205) 2023-08-11 03:31:33,552	INFO trainable.py:927 -- Current state after restoring: {'_iteration': 70, '_timesteps_total': None, '_time_total': 80.51905107498169, '_episodes_total': None} [repeated 2x across cluster]
(_WandbLoggingActor pid=209481) wandb: wandb version 0.15.8